In [20]:
import numpy as np
import random
import gc
import os
import os.path
import re
from keras.models import Sequential
from keras import layers
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras import optimizers
import tensorflow as tf

def file_name(file_dir):
    for root, dirs, files in os.walk(file_dir):
        return files


def data_prepare(file_dir):
    # prepare reads from normal plasma
    files = file_name(file_dir)
    HK = []
    CCR = []
    for file in files:
        if 'HK' in file:
            input_HK = open(file_dir+file,'r')
            for line in input_HK:
                ll = []
                for item in line.strip().split('\t')[:-1]:
                
                    ll.append(round(float(item)))
                HK.append(np.array(ll))
        if 'CCR' in file:
            input_CCR = open(file_dir+file,'r')
            for line in input_CCR:
                ll = []
                for item in line.strip().split('\t')[:-1]:
                    ll.append(round(float(item)))
                CCR.append(np.array(ll))
    return np.array(HK), np.array(CCR)

# transform sequence into one-hot code (0/1/2/3 to one-hot) and add methylation state channel
def conv_onehot(regions):
    # module = np.array([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [0, 0, 0, 1, 0], [0, 0, 1, 0, 1]])
    onehot = np.zeros((len(regions), 19, 100), dtype='int')
    for i in range(len(regions)):
        tmp = regions[i]
        tmp_onehot = np.zeros((19,100), dtype='int')
        for j in range(len(tmp)):
            # print(j, tmp)
            if tmp[j] >= 100:
                k = j
                while k <len(tmp):
                    if tmp[k] < 100:
                        break
                    k += 1
                if k != len(tmp):
                    tmp_onehot[j][tmp[k]] = 1
                else:
                    tmp_onehot[j][-1] = 1 
            else:        
                tmp_onehot[j][tmp[j]] = 1 
            # if tmp[j] == 0:
            #     tmp_onehot[j] = module[0]
            # elif tmp[j] == 1:
            #     tmp_onehot[j] = module[1]
            # elif tmp[j] == 2:
            #     tmp_onehot[j] = module[2]
            # elif tmp[j] == 3:
            #     tmp_onehot[j] = module[3]
            # else:
            #     tmp_onehot[j] = module[4]
        onehot[i] = tmp_onehot
    return onehot


# get the chromosome information and position of each read
def make_chrom_region(chrom_0, region_0):
    i = 0
    chrom = np.zeros(len(chrom_0), dtype='int')
    region = np.zeros(len(region_0), dtype='int')
    while i < len(chrom_0):
        chrom[i] = int(re.findall('(\d+)', chrom_0[i])[0])
        region[i] = region_0[i]
        i = i + 1
    return chrom, region

if __name__ == '__main__':
    # train_dir = '/data/jqli/HCC/12_22_test_program/train_dir/' # directory to store the results and data
    # file_dir = '/data/jqli/HCC/4_1_one_hot_11/train_5_71/' # directory of input data
    train_dir = './train_data/tt/'
    file_dir = './train_data/'

    # preprocess of input data
    # the first column : the chromosome where the read is from (e.g. chr1)
    # the second column : the position of the read on chromosome (int format)
    # the third column of each row in the file should be the sequence of a read with a length of 66bp (A/T/C/G)
    # the fourth column of each row in the file should be the methylation state corresponding to the sequence
    # also with a length of 66bp (1:methylated/0:unmethylated) both columns in the form of 'str'
    HK, CCR = data_prepare(file_dir)
    r = min(len(HK),len(CCR))

    # # randomize before class balance
    perm_0 = random.sample(range(len(HK)), len(HK))
    HK = HK[perm_0]

    HK_3one_hot = conv_onehot(HK[0:r])
    CCR_3one_hot = conv_onehot(CCR[0:r])

In [21]:

    perm = random.sample(range(len(HK[0:r]) + len(CCR[0:r])), len(HK[0:r]) + len(CCR[0:r]))
    data_lstm_all = np.vstack((HK[0:r], CCR[0:r]))
    data = np.vstack((HK_3one_hot[0:r], CCR_3one_hot[0:r]))
    label_all = np.array([1] * len(HK[0:r]) + [0] * len(CCR[0:r]))  # generating labels

    data = data[perm]
    data_lstm_all = data_lstm_all[perm]
    label_all = label_all[perm]

In [22]:
    train_num = int(len(data) * 0.6) # 80% as training set, 75% among them for training, 25% among them for validation
    test_num = int(len(data) * 0.2)
    train_data = data[0:train_num]
    train_label = label_all[0:train_num]
    test_data = data[train_num:(train_num + test_num)]
    test_label = label_all[train_num:(train_num + test_num)]

    data_lstm = data_lstm_all[0:(train_num + test_num)]
    label = label_all[0:(train_num + test_num)]

    data_origin = data_lstm_all[(train_num + test_num)::]  # 20% reserved for other analysis such as motif analysis
    label_origin = label_all[(train_num + test_num)::]

    # np.savetxt(train_dir + 'data_origin_lstm.txt', data_origin)
    # np.savetxt(train_dir + 'label_origin.txt', label_origin)
    # np.savetxt(train_dir + 'chrom_origin.txt', chrom_origin)
    # np.savetxt(train_dir + 'region_origin.txt', region_origin)

In [23]:
    # from keras import optimizers.
    def DISMIR_deep():
        model = Sequential()
        model.add(layers.Convolution1D(filters=100,kernel_size=10,input_shape=(18,100),activation='relu',strides=1,padding='same' ) )
        # model.add(layers.Convolution1D(input_shape=(66, 5),
        #                             nb_filter=100,
        #                             filter_length=10,
        #                             border_mode="same",
        #                             activation="relu",
        #                             subsample_length=1))
        # model.add(layers.MaxPooling1D(pool_length=2, stride=2))
        model.add(layers.MaxPooling1D(pool_size=2,strides=2) )
        model.add(layers.Dropout(0.2))
        model.add(layers.Bidirectional(layers.LSTM(9, return_sequences=True)))
        # model.add(layers.Convolution1D(input_shape=(33, 132),
        #                             nb_filter=100,
        #                             filter_length=3,
        #                             border_mode="same",
        #                             activation="relu",
        #                             subsample_length=1))
        model.add(layers.Convolution1D(filters=100, kernel_size=3, input_shape=(9,18), activation='relu',strides=1,padding='same'))
        # model.add(layers.MaxPooling1D(pool_length=2, stride=2))
        model.add(layers.MaxPooling1D(pool_size=2, strides=2))
        model.add(layers.Dropout(0.2))
        model.add(layers.Flatten())
        model.add(layers.Dense(750, activation='relu', kernel_regularizer=None, bias_regularizer=None))
        model.add(layers.Dropout(0.2))
        model.add(layers.Dense(300, activation='relu', kernel_regularizer=None, bias_regularizer=None))
        model.add(layers.Dense(1, activation='sigmoid', kernel_regularizer=None, bias_regularizer=None))
        return model

In [24]:
    gpus = tf.config.experimental.list_physical_devices(device_type='GPU')
    for gpu in gpus:
        tf.config.experimental.set_memory_growth(gpu, True)
    model = DISMIR_deep()
    model.compile(optimizer='sgd', loss='binary_crossentropy', metrics=['accuracy'])
    early_stopping = EarlyStopping(monitor='val_loss', patience=10)
    # print(train_data.shape)
    print(model.summary())
    history = model.fit(train_data, train_label, epochs=20, batch_size=128, validation_split=0.1,
                        callbacks=[EarlyStopping(patience=10), ModelCheckpoint(filepath=train_dir + 'weight.h5', save_best_only=True)],
                        shuffle=True, verbose=2)


    # model.load_weights(train_dir + 'weight.h5')
    # result = model.predict(test_data, verbose=0)
    # np.savetxt(train_dir + 'test_label.txt', test_label)
    # np.savetxt(train_dir + 'predict_result.txt', result)
    # np.savetxt(train_dir + 'test_chrom.txt', test_chrom)
    # np.savetxt(train_dir + 'test_region.txt', test_region)

    # result_all = model.predict(data[0:(train_num + test_num)], verbose=0)
    # np.savetxt(train_dir + 'data_lstm.txt', data_lstm)
    # np.savetxt(train_dir + 'label_all.txt', label)
    # np.savetxt(train_dir + 'chrom_all.txt', chrom)
    # np.savetxt(train_dir + 'region_all.txt', region)
    # np.savetxt(train_dir + 'result_all.txt', result_all)
    


Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_4 (Conv1D)            (None, 18, 100)           100100    
_________________________________________________________________
max_pooling1d_4 (MaxPooling1 (None, 9, 100)            0         
_________________________________________________________________
dropout_6 (Dropout)          (None, 9, 100)            0         
_________________________________________________________________
bidirectional_2 (Bidirection (None, 9, 18)             7920      
_________________________________________________________________
conv1d_5 (Conv1D)            (None, 9, 100)            5500      
_________________________________________________________________
max_pooling1d_5 (MaxPooling1 (None, 4, 100)            0         
_________________________________________________________________
dropout_7 (Dropout)          (None, 4, 100)           